# Importing the data to Json file

In [2]:
!pip install praw

In [3]:
import sys
import praw
import json

In [4]:
%load_ext autoreload
%autoreload 1
%aimport redditClient

In [5]:
from redditClient import redditClient
import matplotlib.pyplot as plt


from collections import Counter

In [6]:
# From the author and Social Media and Network Analytics, Lab Notes 
import praw
import json
from datetime import datetime
from collections import defaultdict

reddit = praw.Reddit(
    client_id="qRiibhdquxeSUjfMJ7pWAQ",
    client_secret="0RKnbey-l7uVqiVttlMtT-3QxM5p2Q",
    user_agent="client for SNAM2024"
)

subreddit_names = [
    "running", "BeginnersRunning", "beginnerrunning",
    "Marathon_Training", "AdvancedRunning", "XXRunning", "firstmarathon"
]
query = "mental"
num_posts = 500

mental_keywords = ["mental", "anxiety", "depression", "stress", "burnout", "therapy", "psych", "mind", "panic", "emotion", "feel", "mood"]

def contains_mental_keywords(text):
    return any(keyword in text.lower() for keyword in mental_keywords)

data_with_submissions = {"submissions": []}
subreddit_post_counts = defaultdict(int)

for subreddit_name in subreddit_names:
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.search(query, sort="hot", time_filter="all", limit=num_posts)

    for post in posts:
        post_data = {
            "subreddit": subreddit_name,
            "title": post.title,
            "score": post.score,
            "created": post.created_utc,
            "comments": []
        }

        
        post.comments.replace_more(limit=None)
        for top_comment in post.comments[:]:
            if contains_mental_keywords(top_comment.body):
                post_data["comments"].append({
                    "body": top_comment.body,
                    "score": top_comment.score
                })

        if post_data["comments"]:
            data_with_submissions["submissions"].append(post_data)
            subreddit_post_counts[subreddit_name] += 1

# Save to JSON
with open("Running_for_MentalHealth.json", "w", encoding="utf-8") as f:
    json.dump(data_with_submissions, f, indent=4, ensure_ascii=False)

print(f"Saved {len(data_with_submissions['submissions'])} total submissions with relevant comments.")
print("Breakdown by subreddit:")
for name, count in subreddit_post_counts.items():
    print(f"  - {name}: {count} posts")

Saved 929 total submissions with relevant comments.
Breakdown by subreddit:
  - running: 127 posts
  - BeginnersRunning: 38 posts
  - beginnerrunning: 141 posts
  - Marathon_Training: 158 posts
  - AdvancedRunning: 155 posts
  - XXRunning: 198 posts
  - firstmarathon: 112 posts
